In [ ]:
if True:
    %rm -rf source
    !git clone https://github.com/HaiDang2001VN/albert-imdb.git source
    %pip install -U wandb transformers evaluate huggingface_hub accelerate
    colab = True
else:
    print("Not running on Google Colab, skip this cell!")
    colab = False

Cloning into 'source'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 224 (delta 116), reused 217 (delta 111), pack-reused 0
Receiving objects: 100% (224/224), 2.52 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (116/116), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
import json
from pprint import pprint

# Load path to pre-processed data in configs folder
config_file = "vlsp_colab" if colab else "vlsp_local"
with open(f"configs/{config_file}.json") as f:
    configs = json.load(f)

pprint(configs)

{'model_path': {'albert': {'base': 'albert-base-v2',
                           'large': 'albert-large-v2'},
                'bart': {'base': 'facebook/bart-base',
                         'large': 'facebook/bart-large'},
                'electra': {'base': 'google/electra-base-discriminator',
                            'large': 'google/electra-large-discriminator',
                            'small': 'google/electra-small-discriminator'},
                'phobert': {'base': 'vinai/phobert-base',
                            'large': 'vinai/phobert-large'}},
 'path': '/content/source'}


In [ ]:
import wandb

run_model = ["phobert", "base", "format0"]
wandb.init(project="advanced_ai_imdb_dataset", name=f"{run_model[0]}-{run_model[1]}-run-vlsp-{run_model[2]}")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from datasets import load_dataset

source_path = configs["path"]
format_name = run_model[2]
# format_name = "format1"
# train_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/train.jsonl", split="train"
# )
# val_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/dev.jsonl", split="train"
# )
# test_set = load_dataset(
#     "json", data_files=f"./vlsp_preprocessed/{format_name}/test.jsonl", split="train"
# )
dataset = load_dataset(
    "json",
    data_files={
        "train": f"{source_path}/vlsp_preprocessed/{format_name}/train.jsonl",
        "val": f"{source_path}/vlsp_preprocessed/{format_name}/dev.jsonl",
        "test": f"{source_path}/vlsp_preprocessed/{format_name}/test.jsonl",
    },
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
model_path = configs['model_path'][run_model[0]][run_model[1]]
print(model_path)

vinai/phobert-base


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=4,
                                                           output_hidden_states=False
                                                           )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_token(example):
    return tokenizer(
        example["sentence"], padding="max_length", truncation=True, return_tensors="pt"
    )

In [ ]:
tokenized_dataset = dataset.map(preprocess_token, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/599 [00:00<?, ? examples/s]

Map:   0%|          | 0/1437 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(f"Type of {type(predictions)=}")
    print(f"Type of {type(labels)=}")
    print(f"Shape of {predictions[0].shape}")
    print(f"Shape of {predictions[1].shape}")
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    predictions = np.argmax(predictions, axis=1)
    # predictions = np.argmax(predictions.reshape(-1, predictions.shape[-1]), axis=1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        "f1_macro": f1.compute(predictions=predictions, references=labels, average="macro")['f1'],
        "f1_micro": f1.compute(predictions=predictions, references=labels, average="micro")['f1'],
        "f1_weighted": f1.compute(predictions=predictions, references=labels, average="weighted")['f1'],
        "precision_macro": precision.compute(predictions=predictions, references=labels, average="macro")['precision'],
        "precision_micro": precision.compute(predictions=predictions, references=labels, average="micro")['precision'],
        "precision_weighted": precision.compute(predictions=predictions, references=labels, average="weighted")['precision'],
        "recall_macro": recall.compute(predictions=predictions, references=labels, average="macro")['recall'],
        "recall_micro": recall.compute(predictions=predictions, references=labels, average="micro")['recall'],
        "recall_weighted": recall.compute(predictions=predictions, references=labels, average="weighted")['recall'],
    }

In [ ]:
id2label = {0: 'AFFILIATION', 1: 'PART – WHOLE', 2: 'LOCATED', 3: 'PERSONAL - SOCIAL'}
label2id = {value: key for key, value in id2label.items()}
print(label2id)

{'AFFILIATION': 0, 'PART – WHOLE': 1, 'LOCATED': 2, 'PERSONAL - SOCIAL': 3}


In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="albert_imdb",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    save_steps=0.1,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    save_total_limit=1,
    evaluation_strategy="steps",
    eval_steps=0.1,
    logging_steps=0.05,
    report_to="wandb",
    gradient_accumulation_steps=5,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
tokenized_dataset["train"][0]

{'sentence': '<location1/> <sep> <location2/> <sep> Trong ảnh : Nghệ_thuật tạo hoa_văn trên trang_phục truyền_thống của người Mông hoa , tại <location2/> , <location1/> .',
 'label': 2,
 'input_ids': [0,
  63117,
  2029,
  1395,
  48303,
  3123,
  884,
  63117,
  3657,
  1487,
  884,
  63117,
  2029,
  1395,
  48303,
  4937,
  884,
  63117,
  3657,
  1487,
  884,
  92,
  284,
  27,
  5107,
  199,
  9279,
  34,
  1390,
  743,
  7,
  18,
  6275,
  389,
  4,
  35,
  63117,
  2029,
  1395,
  48303,
  4937,
  884,
  4,
  63117,
  2029,
  1395,
  48303,
  3123,
  884,
  5,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision Macro,Precision Micro,Precision Weighted,Recall Macro,Recall Micro,Recall Weighted
63,1.062500,0.365776,0.883139,0.670055,0.883139,0.865347,0.655595,0.883139,0.849770,0.686730,0.883139,0.883139
126,0.297200,0.244657,0.929883,0.940496,0.929883,0.929793,0.943668,0.929883,0.933171,0.940063,0.929883,0.929883
189,0.197800,0.219874,0.939900,0.950381,0.939900,0.939809,0.952237,0.939900,0.939918,0.948679,0.939900,0.939900
252,0.130600,0.213466,0.943239,0.952884,0.943239,0.943409,0.953043,0.943239,0.945663,0.954581,0.943239,0.943239
315,0.083700,0.198261,0.949917,0.959324,0.949917,0.950048,0.959521,0.949917,0.950846,0.959690,0.949917,0.949917
378,0.057900,0.205687,0.941569,0.952097,0.941569,0.941719,0.952581,0.941569,0.942657,0.952266,0.941569,0.941569
441,0.046800,0.220737,0.939900,0.950247,0.939900,0.939931,0.953395,0.939900,0.941731,0.948395,0.939900,0.939900
504,0.028600,0.218044,0.949917,0.958831,0.949917,0.950033,0.959415,0.949917,0.950798,0.958772,0.949917,0.949917


Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


TrainOutput(global_step=504, training_loss=0.19628534903601993, metrics={'train_runtime': 1871.0986, 'train_samples_per_second': 26.722, 'train_steps_per_second': 0.334, 'total_flos': 5304414126735360.0, 'train_loss': 0.19628534903601993, 'epoch': 20.16})

In [ ]:
trainer.evaluate()

Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


{'eval_loss': 0.19826138019561768,
 'eval_accuracy': 0.9499165275459098,
 'eval_f1_macro': 0.959324070831378,
 'eval_f1_micro': 0.9499165275459098,
 'eval_f1_weighted': 0.9500482311172407,
 'eval_precision_macro': 0.9595208905430004,
 'eval_precision_micro': 0.9499165275459098,
 'eval_precision_weighted': 0.9508460320540499,
 'eval_recall_macro': 0.959689565199871,
 'eval_recall_micro': 0.9499165275459098,
 'eval_recall_weighted': 0.9499165275459098,
 'eval_runtime': 14.2302,
 'eval_samples_per_second': 42.094,
 'eval_steps_per_second': 2.67,
 'epoch': 20.16}

In [ ]:
trainer.evaluate(tokenized_dataset["test"], metric_key_prefix="test")

early stopping required metric_for_best_model, but did not find eval_loss so early stopping is disabled


Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)


{'test_loss': 2.970613718032837,
 'test_accuracy': 0.42588726513569936,
 'test_f1_macro': 0.5097113184486626,
 'test_f1_micro': 0.4258872651356994,
 'test_f1_weighted': 0.4305925594583278,
 'test_precision_macro': 0.5136978010542932,
 'test_precision_micro': 0.42588726513569936,
 'test_precision_weighted': 0.4390112327643566,
 'test_recall_macro': 0.5089274459466493,
 'test_recall_micro': 0.42588726513569936,
 'test_recall_weighted': 0.42588726513569936,
 'test_runtime': 34.1979,
 'test_samples_per_second': 42.02,
 'test_steps_per_second': 2.632,
 'epoch': 20.16}

In [ ]:
predictions = trainer.predict(tokenized_dataset["test"]).label_ids
print(predictions.shape)

Type of type(predictions)=<class 'numpy.ndarray'>
Type of type(labels)=<class 'numpy.ndarray'>
Shape of (4,)
Shape of (4,)
(1437,)


In [ ]:
# Showing the sentence, label and predicted label
for i in range(10):
    print(f"====================================== Sentence {i} ======================================")
    print(tokenized_dataset["test"][i]["sentence"])

    gt = tokenized_dataset["test"][i]["label"]
    pred = predictions[i]

    print(f"GT: {gt} - {id2label[gt]}")
    print(f"Pred: {pred} - {id2label[pred]}")

====================================== Sentence 0 ======================================
<organization1/> <sep> <person1/> <sep> Gắn_kết tín_dụng chính_sách với tín_dụng thương_mại để hộ nghèo thoát nghèo bền_vững Đó là chỉ_đạo của ông <person1/> – Phó Thống_đốc <organization1/> – trong Hội_thảo khoa_học Quốc_gia “ Ngân_hàng Chính_sách_xã_hội : 15 năm một chặng đường ” vừa diễn ra ngày 22/9/2017 tại Hà_Nội .
GT: 0 - AFFILIATION
Pred: 0 - AFFILIATION
====================================== Sentence 1 ======================================
<location1/> <sep> <location2/> <sep> Những tín_hiệu vui Hơn 10 ngày trước , tại <location2/> <location1/> , tỉnh Long_An ) đã diễn ra lễ công_bố xuất_khẩu lô hàng thịt gà sạch đầu_tiên của Việt_Nam sang thị_trường Nhật_Bản và lễ ký_kết hợp_tác chiến_lược giữa chuỗi liên_kết sản_xuất thịt gà sạch De_Heus và Cảng Quốc_tế Long_An .
GT: 1 - PART – WHOLE
Pred: 1 - PART – WHOLE
====================================== Sentence 2 ===============================

In [25]:
# Convert to pandas for easier visualization
import pandas as pd
test_df = pd.DataFrame({"sentence": [x["sentence"] for x in tokenized_dataset["test"]],
                        "label": [x["label"] for x in tokenized_dataset["test"]],
                        "prediction": predictions})
test_df.to_csv("test.csv", index=False)
test_df

,sentence,label,prediction
0,<organization1/> <sep> <person1/> <sep> Gắn_kế...,0,0
1,<location1/> <sep> <location2/> <sep> Những tí...,1,1
2,<location1/> <sep> <location2/> <sep> đến Bệnh...,1,1
3,<location1/> <sep> <location2/> <sep> Vào thời...,1,1
4,"<person1/> <sep> <person2/> <sep> <person2/> ,...",3,3
...,...,...,...
1432,<location1/> <sep> <location2/> <sep> Người_dâ...,1,1
1433,<organization1/> <sep> <person1/> <sep> Nhưng ...,0,0
1434,<location1/> <sep> <location2/> <sep> Đó là tâ...,1,1
1435,<location1/> <sep> <organization1/> <sep> Ông ...,1,1


In [ ]:
wandb.finish()

eval/accuracy,▃▁▆███▇█
eval/f1_macro,▅▁▇█████
eval/f1_micro,▃▁▆███▇█
eval/f1_weighted,▄▁▇█████
eval/loss,█▇▂▁▂▃▄▁
eval/precision_macro,▂▁▆██▇▇█
eval/precision_micro,▃▁▆███▇█
eval/precision_weighted,▃▁▆███▇█
eval/recall_macro,▆▁▇█████
eval/recall_micro,▃▁▆███▇█
eval/recall_weighted,▃▁▆███▇█
